<a href="https://colab.research.google.com/github/YashDeshpande1771/Review-Summarization/blob/master/TextRank_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

Code for generating word and Sentence embeddings

In [0]:
#Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

Instructions for updating:
Colocations handled automatically by placer.


W0410 10:07:45.882799 140241228052352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Generate embeddings for word, sentence and review. We can use this to improve feature extraction



In [0]:
word = "Battery"
sentence = "The battery does not last long."
paragraph = (
    "The display is excellent and it is affordable."
    "Although I had some issues with the battery."
    "The phone is overall worth the price.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Battery
Embedding size: 512
Embedding: [-0.028411904349923134, -0.008577127009630203, -0.03163396194577217, ...]

Message: The battery does not last long.
Embedding size: 512
Embedding: [0.035201992839574814, 0.058463502675294876, -0.022710423916578293, ...]

Message: The display is excellent and it is affordable.Although I had some issues with the battery.The phone is overall worth the price.
Embedding size: 512
Embedding: [0.06568684428930283, -0.004557429347187281, -0.02959667332470417, ...]



In [0]:
messages = [
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    "The phone is slow?",
    "the movies are great",
    "battery life is not good",

    "Phone has good warranty",
    "phone is costly",
    "some irrelevant text?",

    "this phone sucks",
    "the screen size of the phone is less",
]

In [0]:


def output_similarity (labels, features, rotation):

  global corr
  corr = np.inner(features, features)
  return (corr)


def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):

  message_embeddings_ = session_.run(

      encoding_tensor, feed_dict={input_tensor_: messages_})
      
  output_similarity(messages_, message_embeddings_, 90)

In [0]:
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  run_and_plot(session, similarity_input_placeholder, messages, similarity_message_encodings)

In [0]:
import networkx as nx
nx_graph = nx.from_numpy_array(corr)
scores = nx.pagerank(nx_graph)
print(scores)

{0: 0.09875412117129174, 1: 0.102721825554344, 2: 0.08189273645236264, 3: 0.10105146356426309, 4: 0.059260922008092215, 5: 0.0960632112267486, 6: 0.09297894568252654, 7: 0.09664207237739239, 8: 0.07166806383453775, 9: 0.10442341378339784, 10: 0.09454322434504311}


In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(messages)), reverse=True)

In [0]:
print(ranked_sentences)

[(0.10442341378339784, 'this phone sucks'), (0.102721825554344, 'My phone is not good.'), (0.10105146356426309, 'The phone is slow?'), (0.09875412117129174, 'I like my phone'), (0.09664207237739239, 'phone is costly'), (0.0960632112267486, 'battery life is not good'), (0.09454322434504311, 'the screen size of the phone is less'), (0.09297894568252654, 'Phone has good warranty'), (0.08189273645236264, 'Your cellphone looks great.'), (0.07166806383453775, 'some irrelevant text?'), (0.059260922008092215, 'the movies are great')]
